# Problem 1(a).  A collectible card class.

You are tasked with organizing information about a Magic: The Gathering card collection.  You are particularly interested in which cards are most rare, and therefore most valuable. Since the game is often played with decks that focus on a particular color or two, you are also interested in the colors of the cards. 

The MTG JSON website (http://mtgjson.com/) has information about different sets of Magic: The Gathering cards in JSON format.  Using the [Example Card](http://mtgjson.com/#exampleCard) information (scroll down if you don't see it!), create a MagicCard class to store relevant information about Magic cards in Python.  

The main features we are interested in are name, colors, and rarity.  Cards may have multiple colors: i.e. a Blue Black card is both blue AND black.  "Colorless" is not a color.

Consider:

* How should someone initialize an instance of your class?
* What functions should your class have?

Your class may have multiple methods for initialization, but at least one of the should accept the MTG JSON represntation of a card as input.

In [1]:
import json
# MTG example json saved in exampleCard.json
data = json.load(open('exampleCard.json'), strict = False)

In [2]:
data

{'artist': 'Greg Staples',
 'cmc': 5,
 'colors': ['White', 'Blue', 'Black'],
 'flavor': 'They are the masters of your mind.',
 'id': '3129aee7f26a4282ce131db7d417b1bc3338c4d4',
 'imageName': 'sen triplets',
 'layout': 'normal',
 'manaCost': '{2}{W}{U}{B}',
 'multiverseid': 180607,
 'name': 'Sen Triplets',
 'number': '109',
 'power': '3',
 'rarity': 'Mythic Rare',
 'subtypes': ['Human', 'Wizard'],
 'supertypes': ['Legendary'],
 'text': "At the beginning of your upkeep, choose target opponent.\n\t         This turn, that player can't cast spells or activate\n\t         abilities and plays with his or her hand revealed.\n\t         You may play cards from that player's hand this turn.",
 'toughness': '3',
 'type': 'Legendary Artifact Creature — Human Wizard',
 'types': ['Artifact', 'Creature']}

In [3]:
class MTG_card(object):
    # constructors
    def __init__(self, name = "", colors = [], rarity = "", power = ''):
        '''default constructor'''
        self.name = name
        self.colors = colors
        self.rarity = rarity
        self.power = power
    
    @classmethod
    def from_json(cls, jsonFile):
        '''construct from json'''
        name = jsonFile['name']
        if 'colors' in jsonFile.keys():
            colors = jsonFile['colors']
        else: 
            colors = None
        if 'power' in jsonFile.keys():
            power = jsonFile['power']
        else:
            power = None
            
        rarity = jsonFile['rarity']
        
        return cls(name, colors, rarity, power)
    
    # getters
    def get_info(self):
        '''return all info'''
        d = dict({'Name: ':self.name, 
                'Colors: ': self.colors, 
                'Rarity: ': self.rarity,
                'Power: ': self.power})
        return d
    
    def get_name(self):
        return self.name
    
    def get_colors(self):
        return self.colors
    
    def get_rarity(self):
        return self.rarity
    
    def get_power(self):
        return self.power
    
    # setters
    def set_from_json(self, jsonFile):
        '''set all info from a json'''
        self.name = jsonFile['name']
        self.colors = jsonFile['colors']
        self.rarity = jsonFile['rarity']
        self.power = jsonFile['power']       
    
    def set_name(self, name):
        self.name = name
    
    def set_colors(self, colors):
        self.colors = [colors]

    def set_rarity(self, rarity):
        self.rarity = rarity
        
    def set_power(self, power):
        self.power = power

In [4]:
exMTG = MTG_card.from_json(data)
exMTG.get_info()

{'Colors: ': ['White', 'Blue', 'Black'],
 'Name: ': 'Sen Triplets',
 'Power: ': '3',
 'Rarity: ': 'Mythic Rare'}

# Problem 1(b). Class for a set of Magic cards.

Download the data for the Dragons of Tarkir (abbreviated as DTK) set from the [Individual Sets](http://mtgjson.com/#individualSets) section of the MTG JSON page.  Create a MagicCardSet class to store relevant information about a set of Magic cards, and create an instance of the class which contains all of the information about the Dragons of Tarkir set.  Your class should use your MTG Set class should store the relevant MTG Cards as instances of your MTG Card class.

In [5]:
DTK = json.load(open('DTK.json'), strict = False)

In [6]:
import pandas as pd
class MagicCardSet(object):
    
    '''builds a list pf objects of class MTG_card'''
    def __init__(self, cards = ''):
        self.cards = cards
        
    @classmethod
    def from_json(cls, jsonFile):
        '''construct from json'''
        cards = [MTG_card.from_json(x) for x in jsonFile['cards']]
        return cls(cards)
    
    def get_card_names(self):
        return [x.get_name() for x in self.cards]
    
    def rarest_level(self):
        rarity_levels = {'Common':0, 
                         'Uncommon':0,
                         'Rare':0,
                         'Mythic Rare':0}
        df = pd.DataFrame(rarity_levels, index=[0])
        for card in self.cards:
            if card.rarity in ['Basic Land']:
                pass
            else:
                df[card.rarity] = df[card.rarity] + 1

        return df.idxmin(axis=1).values[0]
        
    def get_rarity_cards_colors(self, rarity='Mythic Rare'):
        
        rarest_cards = {}
        rarest = rarity #self.rarest_level()
        
        for card in self.cards:
            if card.get_rarity() == rarest:
                rarest_cards[card.get_name()]=card.get_colors()
        
        return rarest_cards
    
    def common_colors(self, rarity):
        from collections import Counter
        
        def flatten(items):
            for i in items:
                if isinstance(i, (list,tuple)):
                    for j in flatten(i):
                        yield j
                else:
                    yield i

        results = pd.DataFrame(Counter(flatten(dtk_set.get_rarity_cards_colors(rarity=rarity).values())), index=[0])

        if None in results.columns:
            results.drop(None, inplace=True, axis=1)
        
        maxNum = max(results.iloc[0,:])
        results = results.columns[results.iloc[0,:]== maxNum].values
        
        if len(results) > 1:
            print('{} colors are tied with {} counts in the "{}" category'.format(len(results),maxNum,rarity))
        print('Most common colors:')
        return list(results)

In [7]:
dtk_set = MagicCardSet.from_json(DTK)

# Problem 1(c).  Dragons of Tarkir.

The MTG rarity levels are "Common", "Uncommon", "Rare", and "Mythic Rare". What are the rarest cards in the Dragons of Tarkir set (meaning list all card names that are from the least common rarity level)?  What colors are they associated with? You might be able to guess by the rarity level names which rarity level is least common, but we want you to use code to verify which is least common, and then print the card names and colors of those cards.

* For purposes of this question, do not consider "Basic Land" as a rarity level.

Write an MTG Set class method to answer this question, then create a markdown cell which explains your method and conclusions. 

In [8]:
# Rarest rarity level
dtk_set.rarest_level()

'Mythic Rare'

In [9]:
# Rarest Cards and their colors
dtk_set.get_rarity_cards_colors(rarity=dtk_set.rarest_level())

{'Clone Legion': ['Blue'],
 'Deathmist Raptor': ['Green'],
 'Descent of the Dragons': ['Red'],
 'Dragon Whisperer': ['Red'],
 'Dragonlord Atarka': ['Red', 'Green'],
 'Dragonlord Dromoka': ['White', 'Green'],
 'Dragonlord Kolaghan': ['Black', 'Red'],
 'Dragonlord Ojutai': ['White', 'Blue'],
 'Dragonlord Silumgar': ['Blue', 'Black'],
 'Narset Transcendent': ['White', 'Blue'],
 'Ojutai Exemplars': ['White'],
 'Risen Executioner': ['Black'],
 'Sarkhan Unbroken': ['Blue', 'Red', 'Green'],
 'Shaman of Forgotten Ways': ['Green'],
 'Shorecrasher Elemental': ['Blue']}

## Explanation
* The get_rarity_cards_colors() method returns the card name and colors for any rarity category. A separate method confirms the rarest category. I split this to keep the get_rarity_cards_colors() method more general for maximum usability (we can now easily determine name and colors for any of the rarity categories)
* rarest_level() confirms the rarest category
* Thus, get_rarity_cards_colors(rarity=dtk_set.rarest_level()) will return names of the rarest category and their colors

# Problem 1(d). Common color for Uncommon cards

Write a class method or Python function to determine which color is most represented among Uncommon cards.  Remember, "colorless" is not a color, and a card that is Blue Black counts as a Blue card and a Black card.

When deciding between a class method or a python function, what was your reasoning? ("X was easier for me to code" is not a good reason!)

In [10]:
# this should be internal to class since it is specific to this class. Especially since I added some output language 
# that is specific to MTG
dtk_set.common_colors('Common')

5 colors are tied with 19 counts in the "Common" category
Most common colors:


['White', 'Blue', 'Black', 'Red', 'Green']

In [11]:
# take a look at all the rarity categories
rares = ['Common', 'Uncommon', 'Rare', 'Mythic Rare']
for rare in rares:
    print(rare)
    print(dtk_set.common_colors(rare))
    print('\n')

Common
5 colors are tied with 19 counts in the "Common" category
Most common colors:
['White', 'Blue', 'Black', 'Red', 'Green']


Uncommon
5 colors are tied with 15 counts in the "Uncommon" category
Most common colors:
['White', 'Blue', 'Black', 'Red', 'Green']


Rare
2 colors are tied with 13 counts in the "Rare" category
Most common colors:
['White', 'Black']


Mythic Rare
Most common colors:
['Blue']


